## Dataset

What is cpe? https://www.acunetix.com/blog/articles/common-platform-enumeration-cpe-explained/

In [5]:
import pandas as pd
import numpy as np
import json
import datasets
np.random.seed(42)

datapath = '/mnt/data/sonia/honeygan/data/processed/data.csv'
df = pd.read_csv(datapath)
print(df.isna().sum())
df.head()

os                 0
ip_str             0
port               0
module             0
cpe           734580
cpe_count          0
category           0
os_generic         0
single_cpe    824511
dtype: int64


,os,ip_str,port,module,cpe,cpe_count,category,os_generic,single_cpe
0,Ubuntu,43.205.13.243,21,ftp,NaN,0,file_sharing,ubuntu,NaN
1,Ubuntu,43.205.13.243,22,ssh,['cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0...,1,remote_access,ubuntu,cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0.5
2,Ubuntu,43.205.13.243,80,http,['cpe:/a:igor_sysoev:nginx'],1,webserver,ubuntu,cpe:/a:igor_sysoev:nginx
3,Ubuntu,43.205.13.243,443,https,['cpe:/a:igor_sysoev:nginx'],1,webserver,ubuntu,cpe:/a:igor_sysoev:nginx
4,Windows (Build 6.3.9600),206.233.189.205,80,http,['cpe:/a:igor_sysoev:nginx'],1,webserver,windows,cpe:/a:igor_sysoev:nginx


In [6]:
dfs = df[(~df.single_cpe.isna())]
dfs.drop(['cpe', 'cpe_count'], axis=1, inplace=True)
print(dfs.isna().sum(), df.shape)
dfs.head()

os            0
ip_str        0
port          0
module        0
category      0
os_generic    0
single_cpe    0
dtype: int64 (1043945, 9)


/tmp/ipykernel_61803/2783460533.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs.drop(['cpe', 'cpe_count'], axis=1, inplace=True)


,os,ip_str,port,module,category,os_generic,single_cpe
1,Ubuntu,43.205.13.243,22,ssh,remote_access,ubuntu,cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0.5
2,Ubuntu,43.205.13.243,80,http,webserver,ubuntu,cpe:/a:igor_sysoev:nginx
3,Ubuntu,43.205.13.243,443,https,webserver,ubuntu,cpe:/a:igor_sysoev:nginx
4,Windows (Build 6.3.9600),206.233.189.205,80,http,webserver,windows,cpe:/a:igor_sysoev:nginx
6,Windows (Build 6.3.9600),206.233.189.205,8081,https-simple-new,webserver,windows,cpe:/a:igor_sysoev:nginx


In [7]:
parsed=json.loads(dfs.to_json(orient='records'))
data_list=[{'input':str(z), 'output':str(r)} for z, r in zip(dfs.index, parsed)]
data = datasets.Dataset.from_list(data_list)
data[:5]

{'input': ['1', '2', '3', '4', '6'],
 'output': ["{'os': 'Ubuntu', 'ip_str': '43.205.13.243', 'port': 22, 'module': 'ssh', 'category': 'remote_access', 'os_generic': 'ubuntu', 'single_cpe': 'cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0.5'}",
  "{'os': 'Ubuntu', 'ip_str': '43.205.13.243', 'port': 80, 'module': 'http', 'category': 'webserver', 'os_generic': 'ubuntu', 'single_cpe': 'cpe:/a:igor_sysoev:nginx'}",
  "{'os': 'Ubuntu', 'ip_str': '43.205.13.243', 'port': 443, 'module': 'https', 'category': 'webserver', 'os_generic': 'ubuntu', 'single_cpe': 'cpe:/a:igor_sysoev:nginx'}",
  "{'os': 'Windows (Build 6.3.9600)', 'ip_str': '206.233.189.205', 'port': 80, 'module': 'http', 'category': 'webserver', 'os_generic': 'windows', 'single_cpe': 'cpe:/a:igor_sysoev:nginx'}",
  "{'os': 'Windows (Build 6.3.9600)', 'ip_str': '206.233.189.205', 'port': 8081, 'module': 'https-simple-new', 'category': 'webserver', 'os_generic': 'windows', 'single_cpe': 'cpe:/a:igor_sysoev:nginx'}"]}

## Save

In [7]:
data.save_to_disk('/mnt/data/sonia/honeygan/llama_format_feb21.dat')

Saving the dataset (0/1 shards):   0%|          | 0/219434 [00:00<?, ? examples/s]

In [31]:
dfs.reset_index(level=0, inplace=True)
dfs.to_csv('/mnt/data/sonia/honeygan/feb13.csv', index=False)

In [10]:
reloaded_dataset = datasets.load_from_disk('/mnt/data/sonia/honeygan/llama_format_feb13.dat')
reloaded_dataset

Dataset({
    features: ['input', 'output'],
    num_rows: 219434
})

In [30]:
import json

In [29]:
df = pd.DataFrame([json.loads(s.replace("'", '"')) for s in reloaded_dataset[:]['output']])
df.index = reloaded_dataset[:]['input']
df

,os,ip_str,port,module,category,os_generic,single_cpe
3873,Ubuntu,43.205.13.243,22,ssh,remote_access,ubuntu,cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0.5
3124,Ubuntu,43.205.13.243,80,http,webserver,ubuntu,cpe:/a:igor_sysoev:nginx
7251,Ubuntu,43.205.13.243,443,https,webserver,ubuntu,cpe:/a:igor_sysoev:nginx
2921,Windows (Build 6.3.9600),206.233.189.205,80,http,webserver,windows,cpe:/a:igor_sysoev:nginx
4342,Windows (Build 6.3.9600),206.233.189.205,8081,https-simple-new,webserver,windows,cpe:/a:igor_sysoev:nginx
...,...,...,...,...,...,...,...
9495,Synology DiskStation Manager (DSM) 6.2.4-25556,58.109.64.115,5000,http,webserver,synology diskstation manager (dsm),cpe:/a:igor_sysoev:nginx
9536,Synology DiskStation Manager (DSM) 6.2.4-25556,121.7.45.72,5000,http-simple-new,webserver,synology diskstation manager (dsm),cpe:/a:igor_sysoev:nginx
7675,Synology DiskStation Manager (DSM) 6.2.4-25556,122.116.172.173,5000,http-simple-new,webserver,synology diskstation manager (dsm),cpe:/a:igor_sysoev:nginx
8200,Synology DiskStation Manager (DSM) 6.2.4-25556,93.30.217.174,5000,http-simple-new,webserver,synology diskstation manager (dsm),cpe:/a:igor_sysoev:nginx


In [32]:
df.reset_index(level=0, inplace=True)
df.to_csv('/mnt/data/sonia/honeygan/feb13.csv', index=False)